In [1]:
import pandas as pd

In [13]:
import numpy as np
import sys

## 1. 데이터 전처리

In [3]:
selloutData = pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0
1,A60,PRODUCT59,201402,22275.0
2,A60,PRODUCT34,201402,4463.0
3,A60,PRODUCT47,201402,0.0
4,A60,PRODUCT56,201402,23.0


컬럼 타입 조작

In [4]:
# 컬럼 타입 조회하기
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK      int64
QTY         float64
dtype: object

In [11]:
# 컬럼별 타입 바꾸기
selloutData["QTY"] = selloutData.QTY.astype("float")
selloutData["REGIONID"] = selloutData.REGIONID.astype("str")
selloutData["PRODUCT"] = selloutData.PRODUCT.astype("str")
selloutData["YEARWEEK"] = selloutData.YEARWEEK.astype("str")

In [12]:
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK     object
QTY         float64
dtype: object

In [20]:
f1 = sys.maxsize
f1

9223372036854775807

In [25]:
f2=f1+1132213243654900000078888099999999999999999999999999999999999999999123
type(f2) 

int

원하는 컬럼 조회

In [7]:
# 데이터프레임명[ [ ‘컬럼명1’, ‘컬럼명2’ ] ]
# 데이터프레임명.filter(items=[ ‘컬럼명1’, ‘컬럼명2’])

selloutData[["REGIONID","PRODUCT"]].head(2)

,REGIONID,PRODUCT
0,A60,PRODUCT4
1,A60,PRODUCT59


In [8]:
selloutData.filter(items = ["REGIONID","PRODUCT"]).head(2)

,REGIONID,PRODUCT
0,A60,PRODUCT4
1,A60,PRODUCT59


In [9]:
testData = selloutData[["REGIONID", "PRODUCT"]]
testData["KEY"] = testData.REGIONID.str[:] + "_" + testData.PRODUCT.str[:]
testData.head()

##  .str 쓰는 이유는 문자열끼리만 붙일 수 있기 때문.

C:\Users\SMART-15\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,REGIONID,PRODUCT,KEY
0,A60,PRODUCT4,A60_PRODUCT4
1,A60,PRODUCT59,A60_PRODUCT59
2,A60,PRODUCT34,A60_PRODUCT34
3,A60,PRODUCT47,A60_PRODUCT47
4,A60,PRODUCT56,A60_PRODUCT56


## 1-1. 불량 데이터 처리

### - 반품값 정제 (음수값 지우기)

In [10]:
#QTY 마이너스 값을 0으로 변경한 자료를 QTY_NEW 컬럼에 생성하세요 

selloutData["QTY_NEW"] = np.where(selloutData["QTY"]<0,0,\
                                 selloutData["QTY"])
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW
0,A60,PRODUCT4,201402,71.0,71.0
1,A60,PRODUCT59,201402,22275.0,22275.0
2,A60,PRODUCT34,201402,4463.0,4463.0
3,A60,PRODUCT47,201402,0.0,0.0
4,A60,PRODUCT56,201402,23.0,23.0


In [11]:
# 잘 바뀌었나 조회해보기
selloutData[selloutData["QTY"]<0].head()

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW
65,A02,PRODUCT16,201403,-1.0,0.0
338,A08,PRODUCT36,201402,-1.0,0.0
1270,A14,PRODUCT36,201405,-1.0,0.0
1616,A17,PRODUCT60,201402,-1.0,0.0
2035,A39,PRODUCT7,201404,-12.0,0.0


In [12]:
len(selloutData)

124658

### - 데이터 통합 (53주차 제거)

In [13]:
# YEAR, WEEK 컬럼을 생성하고 WEEK 가 52 이하인 데이터만 조회한 후 refinedSelloutData 변수에 담으세요

selloutData["YEAR"] = selloutData.YEARWEEK.str[:4]
selloutData["WEEK"] = selloutData.YEARWEEK.str[4:]
selloutData.head(3)

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,02
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,02
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,02


In [14]:
selloutData["WEEK"] = selloutData.WEEK.astype("int")

In [15]:
selloutData[selloutData["WEEK"]==53].head(3)

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
62075,A64,PRODUCT63,201553,2092.0,2092.0,2015,53
62076,A64,PRODUCT15,201553,80.0,80.0,2015,53
62077,A64,PRODUCT4,201553,1781.0,1781.0,2015,53


In [16]:
# refinedSelloutData = selloutData[(selloutData["WEEK"]<=52)]
refinedSelloutData = selloutData[selloutData.WEEK.astype(int)<=52]

In [17]:
refinedSelloutData.head(3)

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,2
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,2
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,2


In [18]:
refinedSelloutData["WEEK"].max()

52

## 3. 데이터 정렬 및 집계

In [19]:
sortyKey = ["REGIONID","PRODUCT","YEAR"]

In [20]:
sortedData = selloutData.sort_values(sortyKey)
selloutData.head(3)

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,2
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,2
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,2


In [21]:
### 기존 인덱스를 drop 하고 새로 인덱스를 생성
sortedData = sortedData.reset_index(drop=True)
sortedData.head(2)

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A00,PRODUCT34,201404,532.0,532.0,2014,4
1,A00,PRODUCT34,201401,661.0,661.0,2014,1


###   

In [22]:
groupKey3 = ["REGIONID", "PRODUCT","YEAR"]

In [23]:
refinedSelloutData.groupby(groupKey3).mean()[["QTY_NEW"]].head(15)

QTY_NEW
REGIONID PRODUCT   YEAR               
A00      PRODUCT34 2014     275.961538
                   2015      86.634615
                   2016      36.576923
         PRODUCT58 2014       2.673077
                   2015       5.711538
                   2016     307.365385
         PRODUCT59 2014   27418.692308
                   2015   52240.846154
                   2016   28213.173077
A01      PRODUCT1  2014    8622.461538
                   2015    6384.788462
                   2016    2131.076923
         PRODUCT12 2014  169397.826923
                   2015  195479.884615
                   2016  225598.500000

In [24]:
refinedSelloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,2
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,2
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,2
3,A60,PRODUCT47,201402,0.0,0.0,2014,2
4,A60,PRODUCT56,201402,23.0,23.0,2014,2


##### 지역, 상품, 연주차 단위로 정렬하여
##### sortedData 변수에 담으세요

In [25]:
sortedData = selloutData[["REGIONID","PRODUCT","YEAR","QTY_NEW"]]
sortedData.head()

,REGIONID,PRODUCT,YEAR,QTY_NEW
0,A60,PRODUCT4,2014,71.0
1,A60,PRODUCT59,2014,22275.0
2,A60,PRODUCT34,2014,4463.0
3,A60,PRODUCT47,2014,0.0
4,A60,PRODUCT56,2014,23.0


In [26]:
sortKey = ["REGIONID","PRODUCT","YEAR"]
sortedData = sortedData.sort_values(sortyKey)
sortedData.head()

,REGIONID,PRODUCT,YEAR,QTY_NEW
205,A00,PRODUCT34,2014,532.0
298,A00,PRODUCT34,2014,661.0
1757,A00,PRODUCT34,2014,679.0
3125,A00,PRODUCT34,2014,578.0
4313,A00,PRODUCT34,2014,423.0


##### [지역, 상품, 연도 별 집계]
##### sortedData 에서 지역, 상품 단위
##### 판매량(QTY_NEW) 의 평균 및 표준편차 연산 후
##### groupData 변수에 담으세요
##### 이후 컬럼명을 QTY_MEAN, QTY_STD로 변경하세요

In [27]:
groupKey_s = ["REGIONID","PRODUCT",'YEAR']

groupData_s = sortedData.\
groupby(groupKey_s).agg\
(['mean','std'])['QTY_NEW']
groupData_s

mean            std
REGIONID PRODUCT   YEAR                              
A00      PRODUCT34 2014     275.961538     162.179634
                   2015      85.698113      72.858024
                   2016      36.576923      94.897539
         PRODUCT58 2014       2.673077       3.889257
                   2015       7.547170      14.909427
                   2016     307.365385     407.622857
         PRODUCT59 2014   27418.692308   11461.345740
                   2015   52315.679245    8241.607554
                   2016   28213.173077    9583.361555
A01      PRODUCT1  2014    8622.461538    3142.837761
                   2015    6383.811321    2536.927101
                   2016    2131.076923    1835.068015
         PRODUCT12 2014  169397.826923  176743.049878
                   2015  196178.811321  192959.986155
                   2016  225598.500000  223957.833468
         PRODUCT14 2014   12495.769231    6870.540503
                   2015   18725.924528    8938.343800
                   2016   22567.307692    9437.701749
         PRODUCT15 2014    5076.769231    1732.873242
                   2015    4841.132075    1547.532404
                   2016    4359.192308    1194.369914
         PRODUCT16 2014    1208.192308     481.798837
                   2015    1443.471698     903.538357
                   2016    1524.538462     727.927530
         PRODUCT19 2014    2588.711538     830.249651
                   2015    3566.377358     876.621821
                   2016    3845.884615     819.779182
         PRODUCT2  2014     376.961538     198.635439
                   2015   19514.000000   13469.266771
                   2016   48481.326923   16741.522764
...                                ...            ...
A68      PRODUCT58 2014       2.942308       4.980013
                   2015       0.679245       0.872079
                   2016       1.346154       5.190053
         PRODUCT59 2014    5558.750000    3556.830798
                   2015   11486.207547    6669.953040
                   2016    6239.403846    3931.333926
A69      PRODUCT34 2014     234.307692     327.175227
                   2015      88.415094      40.506616
                   2016     177.326923      80.456716
         PRODUCT58 2014       1.192308       1.547147
                   2015       4.283019       4.908303
                   2016       8.365385       7.340537
         PRODUCT59 2014    1570.730769    1288.559054
                   2015    9766.509434    4847.071847
                   2016   13596.500000    3262.771134
A70      PRODUCT34 2014    1455.269231     914.891960
                   2015    2749.641509    1544.003563
                   2016    2280.615385    1595.475320
         PRODUCT58 2014      14.384615      15.682079
                   2015      40.735849      47.077739
                   2016     129.769231      62.637962
         PRODUCT59 2014   27002.326923   21271.358758
                   2015   78455.452830   12572.896493
                   2016  102784.307692   16613.683300
A77      PRODUCT1  2014    2262.653846     914.206071
                   2015    3031.547170     838.391674
                   2016    3375.326923     702.359238
         PRODUCT12 2014    2035.788462     762.881491
                   2015    3545.641509     845.037075
                   2016    4837.153846    1841.810824

[2382 rows x 2 columns]

In [28]:
groupData_s = groupData_s.reset_index()
groupData_s.head()

,REGIONID,PRODUCT,YEAR,mean,std
0,A00,PRODUCT34,2014,275.961538,162.179634
1,A00,PRODUCT34,2015,85.698113,72.858024
2,A00,PRODUCT34,2016,36.576923,94.897539
3,A00,PRODUCT58,2014,2.673077,3.889257
4,A00,PRODUCT58,2015,7.547170,14.909427


In [29]:
groupData_s.columns = ['REGIONID','PRODUCT','YEAR','QTY_MEAN','QTY_STD']
groupData_s.head()

,REGIONID,PRODUCT,YEAR,QTY_MEAN,QTY_STD
0,A00,PRODUCT34,2014,275.961538,162.179634
1,A00,PRODUCT34,2015,85.698113,72.858024
2,A00,PRODUCT34,2016,36.576923,94.897539
3,A00,PRODUCT58,2014,2.673077,3.889257
4,A00,PRODUCT58,2015,7.547170,14.909427


## 4. 데이터전처리 완료데이터 / 분석 집계데이터 합치기

##### refinedSelloutData와 groupData를 
##### [REGIONID, PRODUCT, YEAR] 키로 조인하여 
##### mergedData 변수에 아래와 같이 담으세요

In [30]:
refinedSelloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,2
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,2
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,2
3,A60,PRODUCT47,201402,0.0,0.0,2014,2
4,A60,PRODUCT56,201402,23.0,23.0,2014,2


In [31]:
groupData_s.head()

,REGIONID,PRODUCT,YEAR,QTY_MEAN,QTY_STD
0,A00,PRODUCT34,2014,275.961538,162.179634
1,A00,PRODUCT34,2015,85.698113,72.858024
2,A00,PRODUCT34,2016,36.576923,94.897539
3,A00,PRODUCT58,2014,2.673077,3.889257
4,A00,PRODUCT58,2015,7.547170,14.909427


In [32]:
joinKey = ["REGIONID","PRODUCT","YEAR"]

In [33]:
mergeData = pd.merge(refinedSelloutData, groupData_s,\
        left_on=joinKey, right_on = joinKey,\
        how = "lef")\
        [["REGIONID","PRODUCT","YEARWEEK","YEAR","WEEK","QTY_NEW","QTY_MEAN","QTY_STD"]]

In [34]:
mergeData.head()

,REGIONID,PRODUCT,YEARWEEK,YEAR,WEEK,QTY_NEW,QTY_MEAN,QTY_STD
0,A60,PRODUCT4,201402,2014,2,71.0,29.134615,13.225093
1,A60,PRODUCT59,201402,2014,2,22275.0,22993.923077,4245.672181
2,A60,PRODUCT34,201402,2014,2,4463.0,6077.653846,2180.158232
3,A60,PRODUCT47,201402,2014,2,0.0,0.288462,0.571772
4,A60,PRODUCT56,201402,2014,2,23.0,16.730769,8.005183


# 데이터 정렬 및 집계

### -- 정렬

In [35]:
csData = pd.read_csv("../dataset/customerdata.csv")
csData.head()

,CUSTID,AVERAGEPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
0,A13566,4273.900000,3,6.4,1.679181,Big-Screen-lover
1,A14219,3642.441950,2,4.0,2.682023,Sleeping-dog
2,A15312,3653.884565,2,5.0,3.208202,Sleeping-dog
3,A16605,3713.211107,2,6.6,0.900000,Early-bird
4,B10634,3391.074215,2,4.2,2.453656,Sleeping-dog


In [36]:
# 정렬 첫번째 방법

# sortyKey = ["AVERAGEPRICE"]
# csData.sort_values(sortyKey, inplace=True)
# csData.head()

In [37]:
# 정렬 두번째 방법

stdKey = ["CUSTTYPE","AVERAGEPRICE","PRODUCTAGE"]  # 컬럼들을 변수에 저장

stdData = csData.sort_values(stdKey)               # 지정한 컬럼들의 순서대로 정렬(왼->오)
stdData.head(3)

,CUSTID,AVERAGEPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
58,K16632,2650.9,2,4.5,2.233033,Big-Screen-lover
144,Z12772,3029.6,2,4.9,2.093035,Big-Screen-lover
85,Q15445,3083.7,2,5.0,2.475608,Big-Screen-lover


In [38]:
stdData = stdData.reset_index(drop=True)           # 인덱스 번호 리셋

stdData.head(3)

,CUSTID,AVERAGEPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
0,K16632,2650.9,2,4.5,2.233033,Big-Screen-lover
1,Z12772,3029.6,2,4.9,2.093035,Big-Screen-lover
2,Q15445,3083.7,2,5.0,2.475608,Big-Screen-lover


### -- 집계

In [39]:
# 컬럼을 변수에 저장
groupkey = ["CUSTTYPE","EMI"]                      

# 변수의 컬럼들을 순차적으로 그룹화해서, 다른 컬럼들의 평균값을 변수에 저장
groupData = csData.groupby(groupkey).mean()        
groupData

AVERAGEPRICE  DEVICECOUNT  PRODUCTAGE
CUSTTYPE         EMI                                       
Big-Screen-lover 1     3246.000000     5.000000    1.891624
                 2     3535.317391     5.508696    2.223247
                 3     4111.600000     6.400000    1.838520
Early-bird       1     2390.083736     6.300000    1.000000
                 2     3232.445885     6.460606    0.936364
                 3     3455.966035     6.475000    0.962500
Sleeping-dog     1     3185.918464     3.833333    2.357587
                 2     3504.298670     4.353659    2.724274

In [40]:
# EMI별 최댓값을 구하시오.

emiMax = csData.groupby(["EMI"]).max()
emiMax

,CUSTID,AVERAGEPRICE,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
EMI,,,,,
1,Y12458,3419.462596,6.3,2.994395,Sleeping-dog
2,Z19835,4166.533130,6.9,3.483573,Sleeping-dog
3,Z13534,4296.873743,6.9,2.055707,Early-bird


In [41]:
# CUSTTYPE,EMI별 평균가격의 평균을 구하시오.

ctMean = csData.groupby(["CUSTTYPE","EMI"]).mean()[["AVERAGEPRICE"]]
ctMean

AVERAGEPRICE
CUSTTYPE         EMI              
Big-Screen-lover 1     3246.000000
                 2     3535.317391
                 3     4111.600000
Early-bird       1     2390.083736
                 2     3232.445885
                 3     3455.966035
Sleeping-dog     1     3185.918464
                 2     3504.298670

In [42]:
# 인덱스 풀기 (일반 표로 만들기)

ctMean = ctMean.reset_index()
ctMean

,CUSTTYPE,EMI,AVERAGEPRICE
0,Big-Screen-lover,1,3246.000000
1,Big-Screen-lover,2,3535.317391
2,Big-Screen-lover,3,4111.600000
3,Early-bird,1,2390.083736
4,Early-bird,2,3232.445885
5,Early-bird,3,3455.966035
6,Sleeping-dog,1,3185.918464
7,Sleeping-dog,2,3504.298670


In [43]:
# 컬럼헤더 재정의

ctMean.columns = ["CUSTTYPE", "EMI", "PRICE_MEAN"]
ctMean.head(1)

,CUSTTYPE,EMI,PRICE_MEAN
0,Big-Screen-lover,1,3246.0


##### 두 개의 집계함수값을 구하고 싶을 때

In [44]:
# 평균 및 표준편차 구하기
# EMI와 CUSTTYPE에 따른 PRODUCTAGE 평균, 표준편차 구하기

groupKey2 = ["PRODUCTAGE"]

groupData2=csData.\
groupby(["EMI","CUSTTYPE"]).agg\
(['mean','std'])[(groupKey2)]

groupData2

PRODUCTAGE          
                           mean       std
EMI CUSTTYPE                             
1   Big-Screen-lover   1.891624       NaN
    Early-bird         1.000000       NaN
    Sleeping-dog       2.357587  0.400702
2   Big-Screen-lover   2.223247  0.450981
    Early-bird         0.936364  0.105529
    Sleeping-dog       2.724274  0.393191
3   Big-Screen-lover   1.838520  0.194814
    Early-bird         0.962500  0.108781

In [45]:
# NaN값이 나온 EMI가 1이고 얼리버드인 사람은 한명이 맞을까?

check = csData[(csData.EMI==1) & (csData.CUSTTYPE == "Early-bird") ]
check

,CUSTID,AVERAGEPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
68,N14862,2390.083736,1,6.3,1.0,Early-bird


##### 이동평균 구하기

In [46]:
# center = True ; 위아래 2개의 값 + 나 = 5개의 값의 평균을 나의 값 위치에 출력한다.
# center = True ; 내 밑으로 4개의 값 + 나 = 5개의 값의 평균을 맨 마지막 값 위치에 출력한다.

In [47]:
csData["TEST"] = csData["PRODUCTAGE"].rolling(window = 5, center = True).mean()
csData.head(10)

,CUSTID,AVERAGEPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE,TEST
0,A13566,4273.900000,3,6.4,1.679181,Big-Screen-lover,NaN
1,A14219,3642.441950,2,4.0,2.682023,Sleeping-dog,NaN
2,A15312,3653.884565,2,5.0,3.208202,Sleeping-dog,2.184612
3,A16605,3713.211107,2,6.6,0.900000,Early-bird,2.406109
4,B10634,3391.074215,2,4.2,2.453656,Sleeping-dog,2.049705
5,B16849,3755.263391,2,4.7,2.786665,Sleeping-dog,1.921030
6,B18816,3213.911446,2,6.4,0.900000,Early-bird,2.155029
7,C10639,3516.500000,2,5.2,2.564830,Big-Screen-lover,2.266986
8,C12717,3732.900000,2,5.7,2.069995,Big-Screen-lover,2.332377
9,C15050,4020.173425,2,4.9,3.013440,Sleeping-dog,2.530702
